MATE principles: Model efficiency, Action specificity, Token efficiency, and Environmental safety

Demonstrates MATE principles in a clean, modular form,

Avoids external dependencies like @register_tool or ActionContext, for ease of testing and understanding.



In [26]:
# ✅ Set up: Install necessary packages (only if using Gemini API via Vertex AI or Google's SDK)
# For OpenAI: pip install openai
# For Gemini via PaLM API: pip install google-generativeai

!pip install --quiet openai google-generativeai

import openai
import google.generativeai as genai
import json
from datetime import datetime, timedelta
from google.colab import userdata
import os

# ✅ Set your API keys (use your real keys here)
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")


# ✅ Initialize APIs
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key
genai.configure(api_key=GEMINI_API_KEY)

In [27]:
# ✅ Choose models
def call_openai(prompt, model="gpt-4", temperature=0.3):
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

def call_gemini(prompt, model="gemini-1.5-flash-latest", temperature=0.3):
    model = genai.GenerativeModel(model)
    response = model.generate_content(prompt)
    return response.text.strip()

In [22]:
# ✅ M: Model Efficiency
def extract_contact_info(text: str):
    prompt = f"Extract contact information (name, email, phone) from the following:\n\n{text}\n\nReturn only the JSON object within a markdown code block."
    response_text = call_gemini(prompt)
    # Extract JSON from markdown code block
    json_string = response_text.strip().strip('```json').strip('```')
    return json.loads(json_string)  # fast + cheap

def analyze_technical_doc(doc: str):
    prompt = f"Analyze the following technical documentation and find any contradictions or problems:\n\n{doc}\n\nReturn as JSON."
    return json.loads(call_openai(prompt))  # powerful + deep reasoning

In [28]:


# ✅ A: Action Specificity
def reschedule_meeting(event_id: str, new_start_time: str, duration_minutes: int, organizer_email: str):
    # Dummy calendar event (simulate)
    calendar_event = {"id": event_id, "organizer": "organizer@example.com"}

    if calendar_event["organizer"] != organizer_email:
        return "❌ You can only reschedule meetings you organize."

    new_start = datetime.fromisoformat(new_start_time)
    if new_start < datetime.now():
        return "❌ Cannot schedule meetings in the past."

    return {
        "event_id": event_id,
        "new_start_time": new_start.isoformat(),
        "duration_minutes": duration_minutes,
        "status": "✅ Rescheduled successfully!"
    }


In [5]:
# ✅ T: Token Efficiency
def analyze_sales_efficiently(data: str):
    prompt = f"""Sales Data: {data}
1. Calculate YoY growth
2. Identify top 3 trends
3. Flag any big surprises"""
    return call_openai(prompt, model="gpt-3.5-turbo")  # cheaper + efficient


In [6]:
# ✅ E: Environmental Safety
# Simulate a safe read-only environment
def safe_read_document(doc_name: str):
    docs = {
        "policy.txt": "Always verify invoice before approval.",
        "manual.txt": "Use tool X to process data."
    }
    return docs.get(doc_name, "Document not found.")

In [29]:
# 🧪 Try it out
print("📇 Contact Info Extraction:")
print(extract_contact_info("Hi, I'm Priya. Email: priya@test.com, phone: 555-1234."))

print("\n📈 Sales Analysis (Efficient):")
print(analyze_sales_efficiently("Sales were $50K in 2023 and $70K in 2024."))

print("\n📅 Rescheduling Meeting:")
print(reschedule_meeting("event123", (datetime.now() + timedelta(days=1)).isoformat(), 30, "organizer@example.com"))

print("\n📖 Safe Document Read:")
print(safe_read_document("policy.txt"))

📇 Contact Info Extraction:
{'name': 'Priya', 'email': 'priya@test.com', 'phone': '555-1234'}

📈 Sales Analysis (Efficient):
1. YoY Growth:
YoY Growth = ((Sales in 2024 - Sales in 2023) / Sales in 2023) * 100
YoY Growth = (($70K - $50K) / $50K) * 100
YoY Growth = ($20K / $50K) * 100
YoY Growth = 40%

2. Top 3 Trends:
a. Increase in Sales: The sales have increased from $50K in 2023 to $70K in 2024, showing a positive trend in revenue growth.
b. Market Expansion: The increase in sales could indicate that the company has successfully expanded its market reach or introduced new products/services.
c. Customer Retention: The growth in sales could also be attributed to improved customer retention strategies or increased customer satisfaction.

3. Big Surprises:
a. Rapid Growth: The 40% YoY growth rate is a significant increase in sales and could be considered a big surprise for the company, especially if it was not anticipated or planned for.
b. Competitive Advantage: If the company has outper